In [16]:
import pandas as pd
import os
from datetime import datetime

backdays = 60
minute_bar = 5
stock_code_list = ['300750.SZ', '000858.SZ', '000725.SZ', '002594.SZ', '300059.SZ', '000651.SZ', '300015.SZ', '300760.SZ', '000001.SZ', '000807.SZ']
begin_date = '20220407'
end_date = '20231124'


In [2]:
instruction = r''
input = r''
output = r''

In [3]:
df = pd.read_feather(r'/home/syl/Code/FinRL_Market_Simulator/data/000768.SZ/20231123/snapshot.ft')

filter1 = df['Time'] >= 93000000 
filter2 = df['Time'] <= 113000000
filter3 = df['Time'] >= 130000000 
filter4 = df['Time'] <= 150000000 
df = df[(filter1 & filter2) | (filter3 & filter4)]

df['Time'] = pd.to_datetime(df['Time'], format='%H%M%S%f')

df = df.drop_duplicates(keep='last')
df.set_index('Time').to_csv('origin.csv')


In [30]:


def fill_data(folder_path):
    for date_folder in os.listdir(folder_path):
        if date_folder == '20231122' or date_folder == '20231123' or date_folder == '20231124':
            continue
        date_str = datetime.strptime(date_folder, '%Y%m%d').strftime('%Y-%m-%d')
        begin_time = pd.to_datetime(f'{date_str} 09:30')
        break_time = pd.to_datetime(f'{date_str} 11:30')
        resume_time = pd.to_datetime(f'{date_str} 13:00')
        end_time = pd.to_datetime(f'{date_str} 15:00')
        # 生成一个新的连续的时间序列，间隔为3秒
        time_index1 = pd.date_range(start=begin_time, end=break_time, freq='3S')
        time_index2 = pd.date_range(start=resume_time, end=end_time, freq='3S')
        # 创建一个空的DataFrame，用于存储生成的时间序列
        time_df = pd.DataFrame(time_index1.union(time_index2), columns=['Time'])
        
        origin_file = os.path.join(folder_path, date_folder, 'snapshot.ft')
        df = pd.read_feather(origin_file)
        filter1 = df['Time'] >= 93000000 
        filter2 = df['Time'] <= 113000000
        filter3 = df['Time'] >= 130000000 
        filter4 = df['Time'] <= 150000000 
        df = df[(filter1 & filter2) | (filter3 & filter4)]
        df['Time'] = date_folder + df['Time'].astype(str)
        df['Time'] = pd.to_datetime(df['Time'], format='%Y%m%d%H%M%S%f') 
        # 将原始DataFrame与时间序列DataFrame合并
        # 这样会生成一个新的DataFrame，其中包含原始数据和生成的时间序列
        # 'Time'作为合并的依据，缺失的数据将用上一个时间戳补齐
        result_df = pd.merge(time_df, df, how='left', left_on='Time', right_on='Time').ffill()

        # 由于合并可能会产生重复的时间戳，我们需要去掉这些重复项
        # 假设我们希望保留原始DataFrame中的顺序
        result_df = result_df.sort_values('Time').drop_duplicates().reset_index(drop=True)
        assert(len(result_df) == 4802)
        result_df.to_csv(origin_file.replace('.ft', '.csv'), index=False)
        
fill_data('/home/syl/Code/FinRL_Market_Simulator/data/000768.SZ')

In [22]:
fill_data('/home/syl/Code/FinRL_Market_Simulator/data/000768.SZ')

In [27]:
pd.read_feather('/home/syl/Code/FinRL_Market_Simulator/data/000768.SZ/20231124/snapshot.ft').reset_index(drop=True)

,Time,WindCode,Match,0,0.1,NumTrades,IOPV,0.2,0.3,Volume,...,BidVol7,BidVol8,BidVol9,BidVol10,WeightedAvgAskPrice,WeightedAvgBidPrice,TotalAskVol,TotalBidVol,HighLimited,LowLimited
0,2023-11-24 09:30:00,000768.SZ,22.66,0.0,0.0,35.0,0.0,0.0,0.0,47200.0,...,0.20,0.05,0.31,0.48,23.69,22.22,755700.0,208000.0,24.88,20.36
1,2023-11-24 09:30:03,000768.SZ,22.65,0.0,0.0,67.0,0.0,0.0,0.0,79100.0,...,0.10,0.40,0.03,0.18,23.66,22.20,944600.0,253900.0,24.88,20.36
2,2023-11-24 09:30:06,000768.SZ,22.64,0.0,0.0,109.0,0.0,0.0,0.0,91700.0,...,0.13,0.28,0.96,0.62,23.59,22.21,1054200.0,261500.0,24.88,20.36
3,2023-11-24 09:30:09,000768.SZ,22.64,0.0,0.0,123.0,0.0,0.0,0.0,94900.0,...,0.13,0.28,0.96,0.62,23.59,22.21,1062800.0,261300.0,24.88,20.36
4,2023-11-24 09:30:12,000768.SZ,22.64,0.0,0.0,126.0,0.0,0.0,0.0,95300.0,...,0.28,0.96,0.62,1.71,23.57,22.21,1080700.0,264900.0,24.88,20.36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4797,2023-11-24 14:59:48,000768.SZ,22.33,0.0,0.0,13786.0,0.0,0.0,0.0,6144628.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,24.88,20.36
4798,2023-11-24 14:59:51,000768.SZ,22.33,0.0,0.0,13786.0,0.0,0.0,0.0,6144628.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,24.88,20.36
4799,2023-11-24 14:59:54,000768.SZ,22.33,0.0,0.0,13786.0,0.0,0.0,0.0,6144628.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,24.88,20.36
4800,2023-11-24 14:59:57,000768.SZ,22.33,0.0,0.0,13786.0,0.0,0.0,0.0,6144628.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,24.88,20.36


In [4]:
import pandas as pd
import numpy as np

# 假设df是你的原始DataFrame，'Time'是时间列
# 首先，确保'Time'列是datetime类型

# start_time = '93000000'
# break_time = '113000000'
# resume_time = '130000000'
# end_time = '150000000'
# 找出时间列中的最小和最大时间
start_time = pd.to_datetime('1900-01-01 09:30')
break_time = pd.to_datetime('1900-01-01 11:30')
resume_time = pd.to_datetime('1900-01-01 13:00')
end_time = pd.to_datetime('1900-01-01 15:00')
# 生成一个新的连续的时间序列，间隔为3秒
time_index1 = pd.date_range(start=start_time, end=break_time, freq='3S')
# date_index_filtered = time_index1[~((time_index1 > break_time) & (time_index1 < resume_time))]
time_index2 = pd.date_range(start=resume_time, end=end_time, freq='3S')

# 创建一个空的DataFrame，用于存储生成的时间序列
time_df = pd.DataFrame(time_index1.union(time_index2), columns=['Time'])

# 将原始DataFrame与时间序列DataFrame合并
# 这样会生成一个新的DataFrame，其中包含原始数据和生成的时间序列
# 'Time'作为合并的依据，缺失的数据将用上一个时间戳补齐
result_df = pd.merge(time_df, df, how='left', left_on='Time', right_on='Time').ffill()

# 由于合并可能会产生重复的时间戳，我们需要去掉这些重复项
# 假设我们希望保留原始DataFrame中的顺序
result_df = result_df.sort_values('Time').drop_duplicates()
result_df.set_index('Time').to_csv('result.csv')

print(result_df)

                    Time   WindCode  Match    0  0.1  NumTrades  IOPV  0.2  \
0    1900-01-01 09:30:00  000768.SZ  22.59  0.0  0.0       20.0   0.0  0.0   
1    1900-01-01 09:30:03  000768.SZ  22.59  0.0  0.0       48.0   0.0  0.0   
2    1900-01-01 09:30:06  000768.SZ  22.58  0.0  0.0       62.0   0.0  0.0   
3    1900-01-01 09:30:09  000768.SZ  22.58  0.0  0.0       83.0   0.0  0.0   
4    1900-01-01 09:30:12  000768.SZ  22.59  0.0  0.0       93.0   0.0  0.0   
...                  ...        ...    ...  ...  ...        ...   ...  ...   
4797 1900-01-01 14:59:48  000768.SZ  22.61  0.0  0.0    12967.0   0.0  0.0   
4798 1900-01-01 14:59:51  000768.SZ  22.61  0.0  0.0    12967.0   0.0  0.0   
4799 1900-01-01 14:59:54  000768.SZ  22.61  0.0  0.0    12967.0   0.0  0.0   
4800 1900-01-01 14:59:57  000768.SZ  22.61  0.0  0.0    12967.0   0.0  0.0   
4801 1900-01-01 15:00:00  000768.SZ  22.61  0.0  0.0    12967.0   0.0  0.0   

      0.3     Volume  ...  BidVol7  BidVol8  BidVol9  BidVol10 